In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Conv2D, LeakyReLU, UpSampling2D, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
import matplotlib.pyplot as plt

# ============================== #
#        CONFIGURATIONS          #
# ============================== #
IMG_HEIGHT, IMG_WIDTH = 256, 256
BATCH_SIZE = 8
EPOCHS = 10

# Paths
ground_truth_dir = r"E:\dataset\archive (1)\Ground_truth"
noisy_dir = r"E:\dataset\archive (1)\Noisy_folder"

# ============================== #
#        DATA LOADING            #
# ============================== #
def load_images(path):
    images = []
    for file in os.listdir(path):
        img = cv2.imread(os.path.join(path, file))
        img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
        img = img / 255.0  # Normalize
        images.append(img)
    return np.array(images)

X_train_noisy = load_images(noisy_dir)
X_train_clean = load_images(ground_truth_dir)

# ============================== #
#        CNN DENOISER            #
# ============================== #
def build_cnn():
    inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    x = Conv2D(64, (3, 3), padding="same", activation="relu")(inputs)
    x = Conv2D(64, (3, 3), padding="same", activation="relu")(x)
    x = Conv2D(3, (3, 3), padding="same", activation="sigmoid")(x)  # Ensure same shape
    return Model(inputs, x)

cnn = build_cnn()
cnn.compile(optimizer=Adam(learning_rate=0.001), loss=MeanSquaredError())

# Train CNN
cnn.fit(X_train_noisy, X_train_clean, epochs=10, batch_size=8)
cnn.save("cnn_denoising_model.h5")

# ============================== #
#        GAN MODELS              #
# ============================== #
def build_generator():
    inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    
    x = Conv2D(64, (3, 3), padding="same", activation="relu")(inputs)
    x = Conv2D(64, (3, 3), padding="same", activation="relu")(x)
    x = Conv2D(3, (3, 3), padding="same", activation="sigmoid")(x)  # Ensure same shape
    
    return Model(inputs, x)

def build_discriminator():
    inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    
    x = Conv2D(64, (3, 3), padding="same")(inputs)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(128, (3, 3), padding="same")(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(1, (3, 3), padding="same", activation="sigmoid")(x)
    
    return Model(inputs, x)

generator = build_generator()
discriminator = build_discriminator()

discriminator.compile(optimizer=Adam(0.0002), loss=BinaryCrossentropy())

# ============================== #
#        GAN TRAINING            #
# ============================== #
g_losses = []
d_losses = []

for epoch in range(EPOCHS):
    idx = np.random.randint(0, X_train_noisy.shape[0], BATCH_SIZE)
    noisy_images = X_train_noisy[idx]
    clean_images = X_train_clean[idx]
    
    generated_images = generator.predict(noisy_images)
    
    real_labels = np.ones((BATCH_SIZE, IMG_HEIGHT, IMG_WIDTH, 1))
    fake_labels = np.zeros((BATCH_SIZE, IMG_HEIGHT, IMG_WIDTH, 1))
    
    # Train discriminator
    d_loss_real = discriminator.train_on_batch(clean_images, real_labels)
    d_loss_fake = discriminator.train_on_batch(generated_images, fake_labels)
    d_loss = 0.5 * (d_loss_real + d_loss_fake)
    
    # Train generator
    g_loss = discriminator.train_on_batch(noisy_images, real_labels)

    # Store losses
    g_losses.append(g_loss)
    d_losses.append(d_loss)
    
    print(f"Epoch {epoch+1}/{EPOCHS}, Generator Loss: {g_loss:.4f}, Discriminator Loss: {d_loss:.4f}")

# Save models
generator.save("gan_generator.h5")
discriminator.save("gan_discriminator.h5")






Epoch 1/10
38/38 [==============================] - 248s 6s/step - loss: 0.0513
Epoch 2/10
38/38 [==============================] - 238s 6s/step - loss: 0.0166
Epoch 3/10
38/38 [==============================] - 219s 6s/step - loss: 0.0146
Epoch 4/10
38/38 [==============================] - 204s 5s/step - loss: 0.0133
Epoch 5/10
38/38 [==============================] - 204s 5s/step - loss: 0.0141
Epoch 6/10
38/38 [==============================] - 200s 5s/step - loss: 0.0138
Epoch 7/10
38/38 [==============================] - 204s 5s/step - loss: 0.0135
Epoch 8/10
38/38 [==============================] - 184s 5s/step - loss: 0.0131
Epoch 9/10
38/38 [==============================] - 183s 5s/step - loss: 0.0135
Epoch 10/10
1/1 [==============================] - 1s 1s/step
Epoch 1/10, Generator Loss: 0.7170, Discriminator Loss: 0.7048
1/1 [==============================] - 1s 1s/step
Epoch 2/10, Generator Loss: 0.6968, Discriminator Loss: 0.7036
1/1 [==============================] - 1s 

In [10]:
# ============================== #
#        TEST DENOISING          #
# ============================== #
def test_denoising(image_path):
    img = cv2.imread(image_path) / 255.0
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    denoised_img = generator.predict(img)[0]  # Remove batch dim
    denoised_img = (denoised_img * 255).astype(np.uint8)

    cv2.imwrite("denoised_output.png", denoised_img)
    print("Denoised image saved as denoised_output.png")

test_denoising(r"E:/dataset/archive (1)/Noisy_folder/noisy_144839343_db2326a111_c.jpg")

# ============================== #
#        SALT-PEPPER FILTER      #
# ============================== #
def remove_salt_pepper_noise(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    denoised_img = cv2.medianBlur(img, 3)  # Apply median filter
    cv2.imwrite("denoised_median.png", denoised_img)

remove_salt_pepper_noise(r"E:\dataset\archive (1)\Noisy_folder\noisy_144839343_db2326a111_c.jpg")

1/1 [==============================] - 0s 215ms/step
Denoised image saved as denoised_output.png


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, LeakyReLU, BatchNormalization, UpSampling2D, Dense, Flatten, Reshape
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
import cv2
import matplotlib.pyplot as plt

# Function to load and preprocess images (with salt-and-pepper noise)
def load_and_preprocess_images(image_dir, img_size=(128, 128)):
    images = []
    for filename in os.listdir(image_dir):
        img_path = os.path.join(image_dir, filename)
        img = cv2.imread(img_path)
        if img is None:
            continue  # Skip files that can't be read
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, img_size)
        img = cv2.medianBlur(img, 3)  # Apply median filtering to remove salt-and-pepper noise
        img = img / 255.0  # Normalize
        images.append(img)
    return np.array(images)

# Load noisy and normal (clean) images
low_light_images = load_and_preprocess_images(r"E:\dataset\archive (1)\Noisy_folder")
normal_images = load_and_preprocess_images(r"E:\dataset\archive (1)\Ground_truth")

# CNN-based Feature Extractor
feature_extractor = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Flatten(),
    Dense(256, activation='relu'),
    Reshape((16, 16, 1)),
])

# Generator Model
def build_generator(input_shape=(16, 16, 1)):  # Latent space input
    model = Sequential([
        Dense(128 * 16 * 16, activation='relu', input_shape=input_shape),  # Latent input
        Reshape((16, 16, 128)),
        UpSampling2D(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        UpSampling2D(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        Conv2D(3, (3, 3), activation='sigmoid', padding='same')  # Output shape (128, 128, 3)
    ])
    return model

# Discriminator Model
def build_discriminator(input_shape=(128, 128, 3)):  # Accepts generated image
    model = Sequential([
        Conv2D(64, (3, 3), padding='same', input_shape=input_shape),
        LeakyReLU(alpha=0.2),
        Conv2D(128, (3, 3), padding='same'),
        LeakyReLU(alpha=0.2),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    return model

# GAN Model
def build_gan(generator, discriminator):
    discriminator.trainable = False  # Freeze discriminator during GAN training
    z = tf.keras.Input(shape=(16, 16, 1))  # Latent input shape (16, 16, 1)
    generated_img = generator(z)
    validity = discriminator(generated_img)
    gan = Model(z, validity)
    gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
    return gan

# Compile and build the models
generator = build_generator()
discriminator = build_discriminator()

# Compile Discriminator
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# Build GAN
gan = build_gan(generator, discriminator)

# Training Loop
def train_gan(epochs=50, batch_size=32):
    for epoch in range(epochs):
        idx = np.random.randint(0, low_light_images.shape[0], batch_size)
        imgs_low_light = low_light_images[idx]
        imgs_normal = normal_images[idx]
        
        # Get features from low-light images using the feature extractor
        feature_input = feature_extractor.predict(imgs_low_light)
        
        # Generate images using the generator
        gen_imgs = generator.predict(feature_input)
        
        # Train discriminator on real and generated images
        d_loss_real, d_acc_real = discriminator.train_on_batch(imgs_normal, np.ones((batch_size, 1)))
        d_loss_fake, d_acc_fake = discriminator.train_on_batch(gen_imgs, np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        d_acc = 0.5 * np.add(d_acc_real, d_acc_fake)
        
        # Train GAN on the latent input, pushing the generator to create real-looking images
        g_loss = gan.train_on_batch(feature_input, np.ones((batch_size, 1)))
        
        print(f"Epoch {epoch+1}/{epochs}: D Loss={d_loss:.4f}, D Accuracy={d_acc:.4f}, G Loss={g_loss:.4f}")

train_gan()

# Save the generator model (low-light enhancer)
generator.save("low_light_enhancer.h5")




ValueError: Exception encountered when calling layer "reshape_1" (type Reshape).

total size of new array must be unchanged, input_shape = [16, 16, 32768], output_shape = [16, 16, 128]

Call arguments received by layer "reshape_1" (type Reshape):
  • inputs=tf.Tensor(shape=(None, 16, 16, 32768), dtype=float32)

In [7]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, LeakyReLU, BatchNormalization, UpSampling2D, Dense, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
import cv2
import matplotlib.pyplot as plt

# Function to load and preprocess images (resize to 128x128)
def load_and_preprocess_images(image_dir, img_size=(128, 128)):
    images = []
    for filename in os.listdir(image_dir):
        img_path = os.path.join(image_dir, filename)
        img = cv2.imread(img_path)
        if img is None:
            continue  # Skip files that can't be read
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, img_size)  # Ensure the image is resized to (128, 128)
        img = cv2.medianBlur(img, 3)  # Apply median filtering to remove salt-and-pepper noise
        img = img / 255.0  # Normalize
        images.append(img)
    return np.array(images)

# Load noisy and normal (clean) images
low_light_images = load_and_preprocess_images(r"E:\dataset\archive (1)\Noisy_folder")
normal_images = load_and_preprocess_images(r"E:\dataset\archive (1)\Ground_truth")

# Generator Model
def build_generator(input_shape=(128, 128, 3)):  # Input is a noisy image (128, 128, 3)
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        UpSampling2D(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        UpSampling2D(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        Conv2D(3, (3, 3), activation='sigmoid', padding='same')  # Output clean image
    ])
    return model

# Discriminator Model
def build_discriminator(input_shape=(128, 128, 3)):  # Accepts generated image
    model = Sequential([
        Conv2D(64, (3, 3), padding='same', input_shape=input_shape),
        LeakyReLU(alpha=0.2),
        Conv2D(128, (3, 3), padding='same'),
        LeakyReLU(alpha=0.2),
        Flatten(),
        Dense(1, activation='sigmoid')  # Binary classification (real or fake)
    ])
    return model

# GAN Model
def build_gan(generator, discriminator):
    discriminator.trainable = False  # Freeze discriminator during GAN training
    z = tf.keras.Input(shape=(128, 128, 3))  # Input noisy image (128, 128, 3)
    generated_img = generator(z)
    validity = discriminator(generated_img)
    gan = Model(z, validity)
    gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
    return gan

# Compile and build the models
generator = build_generator()
discriminator = build_discriminator()

# Compile Discriminator
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# Build GAN
gan = build_gan(generator, discriminator)

# Training Loop with accuracy printing
def train_gan(epochs=50, batch_size=32):
    for epoch in range(epochs):
        idx = np.random.randint(0, low_light_images.shape[0], batch_size)
        imgs_low_light = low_light_images[idx]
        imgs_normal = normal_images[idx]
        
        # Train discriminator on real and generated images
        d_loss_real, d_acc_real = discriminator.train_on_batch(imgs_normal, np.ones((batch_size, 1)))
        gen_imgs = generator.predict(imgs_low_light)
        d_loss_fake, d_acc_fake = discriminator.train_on_batch(gen_imgs, np.zeros((batch_size, 1)))
        
        # Average discriminator loss and accuracy
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        d_acc = 0.5 * np.add(d_acc_real, d_acc_fake)
        
        # Train GAN on the noisy images to improve generator
        g_loss = gan.train_on_batch(imgs_low_light, np.ones((batch_size, 1)))  # The generator aims to fool the discriminator
        
        # Print discriminator and generator metrics
        print(f"Epoch {epoch+1}/{epochs}: D Loss={d_loss:.4f}, D Accuracy={d_acc:.4f}, G Loss={g_loss:.4f}")
        print(f"Discriminator - Real Accuracy: {d_acc_real:.4f}, Fake Accuracy: {d_acc_fake:.4f}")

train_gan()

# Save the generator model (low-light enhancer)
generator.save("low_light_enhancer.h5")



ValueError: Exception encountered when calling layer "sequential_13" (type Sequential).

Input 0 of layer "dense_9" is incompatible with the layer: expected axis -1 of input shape to have value 2097152, but received input with shape (None, 33554432)

Call arguments received by layer "sequential_13" (type Sequential):
  • inputs=tf.Tensor(shape=(None, 512, 512, 3), dtype=float32)
  • training=False
  • mask=None

In [1]:
import cv2
import numpy as np

def add_gaussian_noise(image, mean=0, var=0.01):
    sigma = var**0.5
    gaussian = np.random.normal(mean, sigma, image.shape)
    noisy_image = np.clip(image + gaussian, 0, 255).astype(np.uint8)
    return noisy_image

def add_salt_pepper_noise(image, prob=0.05):
    noisy_image = image.copy()
    num_salt = np.ceil(prob * image.size * 0.5)
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
    noisy_image[coords] = 255

    num_pepper = np.ceil(prob * image.size * 0.5)
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
    noisy_image[coords] = 0
    return noisy_image

# Load image
image = cv2.imread('image.jpg', cv2.IMREAD_COLOR)

# Add noise
gaussian_noisy_image = add_gaussian_noise(image)
salt_pepper_noisy_image = add_salt_pepper_noise(image)

# Save noisy images
cv2.imwrite('gaussian_noise.jpg', gaussian_noisy_image)
cv2.imwrite('salt_pepper_noise.jpg', salt_pepper_noisy_image)


AttributeError: 'NoneType' object has no attribute 'shape'